In [1]:
print('hello')

hello


In [ ]:
!pip install mediapipe
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
import glob
origin_data = glob.glob(r"../data/kranok-nv/Videos/*")
print(len(origin_data))

In [ ]:
import tensorflow as tf

In [ ]:
origin_data_violent = glob.glob(r"../input/kranoknv/Videos/Violent_*")
origin_data_normal = glob.glob(r"../input/kranoknv/Videos/Normal_*")
print(len(origin_data_violent))
print(len(origin_data_normal))

In [ ]:
import cv2
import tensorflow as tf
from keras.layers import LSTM, Dense,Dropout
from keras.models import Sequential
import math
import mediapipe as mp
from mediapipe.python.solutions import pose as mp_pose
import torch
from threading import Thread
import pandas as pd
import numpy as np
from threading import Thread

In [ ]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5n')
mp_drawing = mp.solutions.drawing_utils
mp_pose =mp.solutions.pose

Lấy keypoint 1 video

In [ ]:
# index_global = 0
def EachVideo(linkVideo): # This function is used to browse through a video
    X = []
    video_path =r""+ linkVideo
    cap = cv2.VideoCapture(video_path)
#     skipTime  = 0
#     skipFrame = 0
    while True:    
        ret, frame = cap.read()
#         skipTime = skipTime +1
        if not ret:
            break
        if 1==1:
#         if skipTime >= 30: # When skipTime has passed the first 30 frames, that is, the first 1 second, proceed with Detect Person
#             skipFrame = skipFrame +1 # The skipFrame variable means that I will detect 1 time every 5 frames, so in 1 second I will detect 6 times.
            # print(skipFrame)
#             if  skipFrame  == 5: # When skipFrame = 5, I will detect person and assign skipFrame = 0 to run again
            if 1==1:
#                 skipFrame = 0
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame.flags.writeable = False  
                result = yolo_model(frame)     # Detect Person
                frame.flags.writeable = True   
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                for (xmin, ymin, xmax,   ymax,  confidence,  clas) in result.xyxy[0].tolist(): # Loop through all Persons in the video, giving x,y of each Person
                    c_lm = []
                    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
                            
#                             frame.flags.writeable = False  

                            resulta = pose.process(frame[int(ymin):int(ymax),int(xmin):int(xmax):])
#                             frame.flags.writeable = True   

                            if resulta.pose_landmarks and clas ==0: # clas here is class, class = 0 means human
                                for (id, lm) in enumerate(resulta.pose_landmarks.landmark):
                                    if id > 10 and id not in [17,18,19,20,21,22] and id not in [29,30,31,32] :
                                        c_lm.append(lm.x)
                                        c_lm.append(lm.y)
                                        c_lm.append(lm.z)
#                                         c_lm.append(lm.visibility)
                    if len(c_lm) > 0: # c_ lm is used to save the x and y variables of a person in a loop through each person. When saving, the status will be that there is no x,y data to save.
                        X.append(c_lm) # If the Video link is violent, we add data to X_violent
#                 if cv2.waitKey(1) & 0xFF == ord('q'):
#                       break
    return X
    cap.release()
    cv2.destroyAllWindows()

Lặp qua nhiều video để lấy keypoint

In [ ]:
def AllVideo(startt,endd,dataset): # This function is used to browse all videos
# Status (linkVideo) is: violent, non-violent
    X1 = []
    
# cam (camIndex) is: cam1 or cam2
# Video number is: from start to end, for example non-violent data set is startt 1->61 for cam 1, violent from 1 to 115 for cam 1
    for id,i in enumerate(range(startt,endd)):
        print(f'{startt} -> {endd} ## index: {startt+id}')
        X1.extend(EachVideo(dataset[i])) # Implement the function to detect
    return X1

Save the keypoint, run the getData function to run multiple videos at the same time (threading) to get the keypoint

In [ ]:
X = []
def getData(startt,endd,origin_data):
    global X
    X = AllVideo(startt,endd,origin_data)

    
active = []
skipT = 2000
for i in range(0,10):
    active_1 = Thread(target=getData,args=(skipT,skipT+20,origin_data_normal))
    skipT = skipT + 20
    active.append(active_1)

for i in active:
    i.start()
for i in active:
    i.join()

print(len(X))
X = np.array(X)
print(X.shape)

Save keypoints to file

In [ ]:
X = pd.DataFrame(X)
X  = np.nan_to_num(X) # Once there is data, there will be some words in the data that are NaN, so they should be converted to 0
X = pd.DataFrame(X)
X.to_csv("./normal_2000_2500.csv")

**Do not use the part below**

Model 1

In [ ]:
violent_1_200 = pd.read_csv("../input/subkevin/violent_1_200.csv")
violent_200_400 = pd.read_csv("../input/subkevin/violent_200_400.csv")
violent_400_600 = pd.read_csv("../input/subkevin/violent_400_600.csv")

normal_1_500 = pd.read_csv("../input/subkevin/normal_1_500.csv")
normal_500_1000 = pd.read_csv("../input/subkevin/normal_500_1000.csv")
normal_1000_1500 = pd.read_csv("../input/subkevin/normal_1000_1500.csv")
normal_1500_2000 = pd.read_csv("../input/subkevin/normal_1500_2000.csv")
normal_2000_2500 = pd.read_csv("../input/subkevin/normal_2000_2500.csv")

violent_1_200 = violent_1_200.append(violent_200_400)
violent_1_200 = violent_1_200.append(violent_400_600)


normal_1_500 = normal_1_500.append(normal_500_1000)
normal_1_500 = normal_1_500.append(normal_1000_1500)
normal_1_500 = normal_1_500.append(normal_1500_2000)
normal_1_500 = normal_1_500.append(normal_2000_2500)


X = []
y = []

X_violent = violent_1_200.iloc[:,1:].values
len_X_violent = len(X_violent)
for i in range(10,len_X_violent):
    X.append(X_violent[i-10:i,:])
    y.append(1)
    
    
X_non_violent = normal_1_500.iloc[:,1:].values
len_X_non_violent = len(X_non_violent)
for i in range(10,len_X_non_violent):
    X.append(X_non_violent[i-10:i,:])
    y.append(0)
    
    
X = np.array(X)
y = np.array(y)    
print(violent_1_200.shape)
print(normal_1_500.shape)
print(X.shape)
print(y.shape)



In [ ]:
model  = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1],X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(Dense(units = 4, activation="relu"))
model.add(Dense(units = 1, activation="sigmoid"))
model.compile(optimizer="adam",  metrics = ['accuracy'], loss = "binary_crossentropy")

In [ ]:
model.fit(X, y, epochs=32, batch_size=32,validation_split=0.2)

Model 2

In [ ]:
def getAngle(array_a):
    len_a = len(array_a)
    len_x  = int(len_a/2)
    so_chan = 0
    fa = []
    for i in range(0,len_x):
        fa.append(so_chan)
        so_chan = so_chan + 2
        
    ff = []
    dem = 0
    sum  = 0
    for i in range(4,len_a):
        ff.append([])
        for j in range(i-4,i+1):
            
            len_x  = int(len(array_a[j])/2)
            for i in range(0,len_x):
                fa.append(so_chan)
                so_chan = so_chan + 2

            for k in range(0,len_x-1):
                for m in range(k+1,len_x):
                    tru_x =  array_a[j][fa[m]] - array_a[j][fa[k]] 
                    tru_y = array_a[j][fa[m]+1] -  array_a[j][fa[k]+1]
                    if tru_x == 0:
                        tru_x=1
                    atan = math.atan(tru_y / tru_x)
                    ff[dem].append(atan)
                    
        dem = dem + 1
    return ff

In [ ]:
normal_1_500_copy = np.copy(normal_1_500)
violent_1_200_copy = np.copy(violent_1_200)
                             
normal_1_500_array = list(normal_1_500_copy)
violent_1_200_array = list(violent_1_200_copy)

X_model_2_violent  = []
X_model_2_violent.extend(getAngle(violent_1_200_array))
X_model_2_violent = pd.DataFrame(X_model_2_violent)


X_model_2_non_violent = []
X_model_2_non_violent.extend(getAngle(normal_1_500_array))
X_model_2_non_violent = pd.DataFrame(X_model_2_non_violent)


X_2 = list([])
y_2 = list([])
# print(len(X_model_2_violent))

X_model_2_violent =  X_model_2_violent.iloc[:,1:].values
len_X_model_2_violent = len(X_model_2_violent)
for i in range(10,len_X_model_2_violent):
    X_2.append(X_model_2_violent[i-10:i,:])
    y_2.append(1)
    

X_model_2_non_violent =  X_model_2_non_violent.iloc[:,1:].values
len_X_model_2_non_violent = len(X_model_2_non_violent)
for i in range(10,len_X_model_2_non_violent):
    X_2.append(X_model_2_non_violent[i-10:i,:])
    y_2.append(0)


# y_model_2 = []

# len_X_violent = len(violent_1_200_array)
# for i in range(10,len_X_violent):
#     X_model_2.append(X_violent[i-10:i,:])
#     y.append(1)
# X = []
# y = []
X_2 = np.array(X_2)
y_2 = np.array(y_2)

print(X_2.shape)
print(y_2.shape)

In [ ]:
model_3  = Sequential()

model_3.add(Dense( 500, activation="relu",input_shape = (X_2.shape[1],X_2.shape[2])))
model_3.add(Dense( 300, activation="relu"))

# model_3.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1],X.shape[2])))
model_3.add(Dropout(0.4))

model_3.add(Dense( 100, activation="relu"))
model_3.add(Dense( 50, activation="relu"))
model_3.add(Dropout(0.3))

model_3.add(Dense( 25, activation="relu"))
model_3.add(Dense( 10, activation="relu"))
model_3.add(Dense( 5, activation="relu"))
model_3.add(Dense( 1, activation="sigmoid"))
model_3.compile(optimizer="adam",  metrics = ['accuracy'], loss = "binary_crossentropy")
model_3.summary()

In [ ]:
model_3.fit(X_2, y_2, epochs=32, batch_size=32,validation_split=0.2)

Data artlab

In [ ]:
X= []
y =[]
non_violent = pd.read_csv("../input/subkevin/non_violent.csv")
X_non_violent = non_violent.iloc[:,1:].values
len_X_non_violent = len(X_non_violent)
for i in range(10,len_X_non_violent):
  X.append(X_non_violent[i-10:i,:])
  y.append(0)
violent = pd.read_csv("../input/subkevin/violent.csv")
X_violent = violent.iloc[:,1:].values
len_X_violent = len(X_violent)
for i in range(10,len_X_violent):
  X.append(X_violent[i-10:i,:])
  y.append(1)


In [ ]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

In [ ]:
model2  = Sequential()
model2.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1],X.shape[2])))
model2.add(Dropout(0.2))
model2.add(LSTM(units = 50, return_sequences = True))
model2.add(Dropout(0.2))
model2.add(LSTM(units = 50, return_sequences = True))
model2.add(Dropout(0.2))
model2.add(Dense(units = 4, activation="relu"))
model2.add(Dense(units = 1, activation="sigmoid"))
model2.compile(optimizer="adam",  metrics = ['accuracy'], loss = "binary_crossentropy")

In [ ]:
model2.fit(X, y, epochs=32, batch_size=32,validation_split=0.2)

In [ ]:
model.save("./model_v41.h5")